In [38]:
import os
from glob import glob
import pandas as pd
from barplots import barplots
from scipy.stats import wilcoxon

In [39]:
datasets = {
    "resized_processed": "No_processing",
    "processed": "Processed",
    "demosaicking_processed": "Processed_Demosaicking",
}

In [40]:
data = []
for path in glob("results/*/*/*/*.csv"):
    _, dataset, _, model, _ = path.split(os.sep)
    df = pd.read_csv(path, index_col=0)
    df["dataset"] = datasets[dataset]
    df["model"] = model
    data.append(df)
df = pd.concat(data).reset_index(drop=True)
df

,loss,accuracy,AUPRC,AUROC,run_type,holdout_number,task_name,dataset,model
0,0.420109,0.894413,0.447998,0.857338,training,0,covid_or_pneumonia_vs_other,No_processing,ResNet50V2
1,0.762970,0.609436,0.740128,0.545573,validation,0,covid_or_pneumonia_vs_other,No_processing,ResNet50V2
2,0.739079,0.306723,0.208972,0.402826,test,0,covid_or_pneumonia_vs_other,No_processing,ResNet50V2
3,7.748330,0.023170,0.023639,0.529095,training,0,covid_vs_pneumonia,No_processing,ResNet50V2
4,0.897976,0.870301,0.886602,0.530332,validation,0,covid_vs_pneumonia,No_processing,ResNet50V2
...,...,...,...,...,...,...,...,...,...
319,0.922470,0.859023,0.889321,0.539213,validation,5,covid_vs_pneumonia,Processed,InceptionResNetV2
320,3.206536,0.292595,0.099572,0.828164,test,5,covid_vs_pneumonia,Processed,InceptionResNetV2
321,0.387566,0.972859,0.013249,0.795778,training,5,covid_vs_other,Processed,InceptionResNetV2
322,0.819647,0.399738,0.682353,0.582761,validation,5,covid_vs_other,Processed,InceptionResNetV2


In [43]:
for model in set(df.model):
    temp = df[df.model == model]
    if False:
        barplots(
            temp.drop(columns=["holdout_number"]),
            data_label=f"{model} - {{feature}}",
            path=f"barplots/{model}/{{feature}}.png",
            groupby=["run_type", "task_name", "dataset"],
            orientation="horizontal",
            height=5,
            unique_minor_labels=True,
            subplots=True,
            show_legend=True
        )
    for task in set(temp.task_name):
        filtered = temp[(temp.run_type == "validation") & (temp.task_name == task)]
        no_processing = filtered[filtered.dataset == "No_processing"]
        processed_demosaicking = filtered[filtered.dataset == "Processed_Demosaicking"]
        processed = filtered[filtered.dataset == "Processed"]
        print(model, task, "No_processing vs Processed", "AUPRC", wilcoxon(no_processing.AUPRC, processed.AUPRC))
        print(model, task, "No_processing vs Processed Demosaicking", "AUPRC", wilcoxon(no_processing.AUPRC, processed_demosaicking.AUPRC))
        print(model, task, "No_processing vs Processed", "AUROC", wilcoxon(no_processing.AUROC, processed.AUROC))
        print(model, task, "No_processing vs Processed Demosaicking", "AUROC", wilcoxon(no_processing.AUROC, processed_demosaicking.AUROC))
        

ResNet50V2 covid_vs_other No_processing vs Processed AUPRC WilcoxonResult(statistic=8.0, pvalue=0.6875)
ResNet50V2 covid_vs_other No_processing vs Processed Demosaicking AUPRC WilcoxonResult(statistic=4.0, pvalue=0.21875)
ResNet50V2 covid_vs_other No_processing vs Processed AUROC WilcoxonResult(statistic=10.0, pvalue=1.0)
ResNet50V2 covid_vs_other No_processing vs Processed Demosaicking AUROC WilcoxonResult(statistic=4.0, pvalue=0.21875)
ResNet50V2 covid_or_pneumonia_vs_other No_processing vs Processed AUPRC WilcoxonResult(statistic=6.0, pvalue=0.4375)
ResNet50V2 covid_or_pneumonia_vs_other No_processing vs Processed Demosaicking AUPRC WilcoxonResult(statistic=10.0, pvalue=1.0)
ResNet50V2 covid_or_pneumonia_vs_other No_processing vs Processed AUROC WilcoxonResult(statistic=6.0, pvalue=0.4375)
ResNet50V2 covid_or_pneumonia_vs_other No_processing vs Processed Demosaicking AUROC WilcoxonResult(statistic=8.0, pvalue=0.6875)
ResNet50V2 covid_vs_pneumonia No_processing vs Processed AUPRC Wil